In [1]:
# Import the new v3 modules
import sys, os
sys.path.insert(0, os.path.abspath('..'))

from src.map_viewer_v3 import MapViewer
from src.geocode_v3 import geocode_place  # Keep using existing geocode
from src.area_calc_v3 import compute_area  # Keep using existing area calc
from src.config_v3 import (BHUVAN_LAYERS, DEFAULT_CENTER, DEFAULT_ZOOM, 
                          EXTERNAL_BASEMAPS, DEFAULT_BASE_LAYER, DEFAULT_OVERLAYS)

# UI components
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# ---- Initialize map viewer ----
mv = MapViewer(center=DEFAULT_CENTER, zoom=DEFAULT_ZOOM)
m = mv.get_map()

# ---- Active layer tracking ----
active_base = DEFAULT_BASE_LAYER
active_overlays = set(DEFAULT_OVERLAYS)  # Use a set for easier management

# ---- Create UI components ----

# 1. Base layer selector (dropdown)
base_selector = widgets.Dropdown(
    options=[k for k, v in BHUVAN_LAYERS.items() if not v.get('overlay', False)],
    value=active_base,
    description='Base layer:'
)

# 2. Overlay selectors (checkboxes in an accordion)
overlay_checkboxes = {}

for layer_name, layer_info in BHUVAN_LAYERS.items():
    if layer_info.get('overlay', False):
        checkbox = widgets.Checkbox(
            value=layer_name in active_overlays,
            description=layer_name,
            disabled=False
        )
        overlay_checkboxes[layer_name] = checkbox

overlay_accordion = widgets.Accordion(
    children=[widgets.VBox([overlay_checkboxes[k] for k in overlay_checkboxes])],
    selected_index=None
)
overlay_accordion.set_title(0, 'Overlays')

# 3. Place search
search_box = widgets.Text(description='Search:')
search_btn = widgets.Button(description='Go')

# 4. Unit selector for area calculations
unit_selector = widgets.Dropdown(
    options=[('km²', 'km2'), ('ha', 'ha'), ('m²', 'm2')],
    value='km2',
    description='Units:'
)

# 5. Status output for messages and area results
status_out = widgets.Output()
area_out = widgets.Output()

# ---- Event handlers ----

# Base layer change handler
def on_base_change(change):
    global active_base
    if change['name'] == 'value':
        new_base = change['new']
        active_base = new_base
        with status_out:
            clear_output()
            print(f"Loading base layer: {new_base}...")
        update_map_layers()

base_selector.observe(on_base_change, names='value')

# Overlay checkbox handlers
def create_overlay_handler(layer_name):
    def handler(change):
        if change['name'] == 'value':
            is_active = change['new']
            with status_out:
                if is_active:
                    active_overlays.add(layer_name)
                    clear_output()
                    print(f"Adding overlay: {layer_name}")
                else:
                    active_overlays.discard(layer_name)
                    clear_output()
                    print(f"Removing overlay: {layer_name}")
            update_map_layers()
    return handler

for layer_name, checkbox in overlay_checkboxes.items():
    checkbox.observe(create_overlay_handler(layer_name), names='value')

# Function to update all map layers based on current selections
def update_map_layers():
    # Clear existing layers
    mv.clear_all_layers()
    
    # Add the selected base layer
    base_layer_info = BHUVAN_LAYERS[active_base]
    
    if base_layer_info['type'] == 'wms':
        # Use the layer-specific URL if provided
        custom_url = base_layer_info.get('url', None)
        with status_out:
            print(f"Trying to load WMS layer: {base_layer_info['name']} from {custom_url or 'default URL'}...")
        result = mv.add_wms_layer(base_layer_info['name'], is_base=True, wms_url=custom_url)
        if result is None:
            # If WMS failed, fall back to a reliable tile layer
            with status_out:
                print(f"WMS layer failed to load. Falling back to ESRI Satellite...")
            mv.add_external_tile_layer("ESRI Satellite")
    elif base_layer_info['type'] == 'external_tile':
        with status_out:
            print(f"Loading external tile layer: {active_base}...")
        mv.add_external_tile_layer(active_base)
    else:  # Default tile layer
        with status_out:
            print(f"Loading OSM tile layer...")
        mv.add_tile_layer()
    
    # Add all active overlays
    for overlay_name in active_overlays:
        overlay_info = BHUVAN_LAYERS[overlay_name]
        if overlay_info['type'] == 'wms':
            custom_url = overlay_info.get('url', None)
            with status_out:
                print(f"Adding overlay: {overlay_name}...")
            result = mv.add_wms_layer(overlay_info['name'], is_base=False, 
                                    layer_id=overlay_name, wms_url=custom_url)
            if result is None:
                with status_out:
                    print(f"Failed to add overlay: {overlay_name}")

# Search handler
def on_search(b):
    with area_out:
        clear_output()
        with status_out:
            clear_output()
            print(f"Searching for: {search_box.value}...")
        try:
            lat, lon = geocode_place(search_box.value)
            m.center = (lat, lon)
            m.zoom = 12
            with status_out:
                clear_output()
                print(f"Found location: {search_box.value} ({lat}, {lon})")
        except Exception as e:
            with status_out:
                clear_output()
                print(f"Geocoding error: {e}")

search_btn.on_click(on_search)

# Draw handler for area calculation
def handle_draw(target, action, geo_json):
    # Extract coordinates from the first ring
    coords = geo_json['geometry']['coordinates'][0]
    area = compute_area(coords, unit_selector.value)
    with area_out:
        clear_output()
        print(f"Area: {area:.3f} {unit_selector.label}")

mv.on_draw(handle_draw)

# ---- Layout and display ----
controls_top = widgets.HBox([search_box, search_btn, unit_selector])
controls_layers = widgets.VBox([base_selector, overlay_accordion])
outputs = widgets.VBox([status_out, area_out])

# Display the UI
display(HTML("<h3>Bhuvan Map Viewer v3</h3>"))
display(controls_top)
display(controls_layers)
display(m)
display(outputs)

# Initialize with default base layer
with status_out:
    print(f"Initializing map with {active_base}...")
update_map_layers()

Map(center=[20.5937, 78.9629], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…